<a href="https://colab.research.google.com/github/ikasuryani/Sanbercode0920FinalProject/blob/main/Main-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
trainlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/train.csv'
testlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/test.csv'

df = pd.read_csv(trainlink)

#Encoding
df = pd.get_dummies(df, columns=['Jenis Kelamin', 'Status Perkawinan', 'Kelas Pekerja'])
df = pd.get_dummies(df, columns=['Gaji'], drop_first=True)
df = df.rename(columns={'Gaji_>7jt':'Gaji'})
pddterakhir = {
    '10th': 'SMP',
    '11th': 'SMP',
    '7th-8th': 'SD',
    '9th': 'SD',
    '12th': 'SMP',
    '1st-4th': 'TK',
    '5th-6th': 'TK'
}
df['Pendidikan'] = df['Pendidikan'].replace(pddterakhir)
df['Pekerjaan'].replace("?", float("NaN"), inplace=True)
df.dropna(subset=["Pekerjaan"], inplace=True)
df = pd.get_dummies(df, columns=['Pendidikan', 'Pekerjaan'])
df = df.drop(['Berat Akhir', 'Keuntungan Kapital', 'Kerugian Capital'], axis=1)

df.head()

,id,Umur,Jmlh Tahun Pendidikan,Jam per Minggu,Jenis Kelamin_Laki2,Jenis Kelamin_Perempuan,Status Perkawinan_Belum Pernah Menikah,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Kelas Pekerja_?,Kelas Pekerja_Pekerja Bebas Bukan Perusahan,Kelas Pekerja_Pekerja Bebas Perusahaan,Kelas Pekerja_Pemerintah Lokal,Kelas Pekerja_Pemerintah Negara,Kelas Pekerja_Pemerintah Provinsi,Kelas Pekerja_Tanpa di Bayar,Kelas Pekerja_Tidak Pernah Bekerja,Kelas Pekerja_Wiraswasta,Gaji,Pendidikan_D3,Pendidikan_D4,Pendidikan_Doktor,Pendidikan_Master,Pendidikan_Pendidikan Tinggi,Pendidikan_SD,Pendidikan_SMA,Pendidikan_SMP,Pendidikan_Sarjana,Pendidikan_Sekolah Professional,Pendidikan_TK,Pekerjaan_Asisten Rumah Tangga,Pekerjaan_Ekesekutif Managerial,Pekerjaan_Mesin Inspeksi,Pekerjaan_Pembersih,Pekerjaan_Pemuka Agama,Pekerjaan_Penjaga,Pekerjaan_Perbaikan Kerajinan,Pekerjaan_Petani,Pekerjaan_Sales,Pekerjaan_Servis Lainnya,Pekerjaan_Spesialis,Pekerjaan_Supir,Pekerjaan_Tech-support,Pekerjaan_Tentara
0,0,21,9,35.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,49,6,40.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,44,12,45.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,3,24,13,35.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4,33,14,40.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [2]:
# df['Kelas Pekerja'].value_counts()
# mask = df['Pendidikan'].isnull()
# print(df[mask])

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

X = df.drop('Gaji', axis=1)
y = df['Gaji']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.847714 (0.014770)
LDA: 0.874699 (0.007133)
KNN: 0.626102 (0.014046)
CART: 0.695087 (0.006341)
NB: 0.870606 (0.005177)


In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Make predictions on validation dataset
# model = LinearDiscriminantAnalysis()
model = GaussianNB() #best score from kaggle, 0.90xx
# model = LogisticRegression(solver='liblinear', multi_class='ovr')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8105399669889177
[[5958  416]
 [1191  917]]
              precision    recall  f1-score   support

           0       0.83      0.93      0.88      6374
           1       0.69      0.44      0.53      2108

    accuracy                           0.81      8482
   macro avg       0.76      0.68      0.71      8482
weighted avg       0.80      0.81      0.79      8482



In [7]:
dftest = pd.read_csv(testlink)
#Encoding
dftest = pd.get_dummies(dftest, columns=['Jenis Kelamin', 'Status Perkawinan', 'Kelas Pekerja'])
# df = pd.get_dummies(df, columns=['Gaji'], drop_first=True)
# df = df.rename(columns={'Gaji_>7jt':'Gaji'})
pddterakhir = {
    '10th': 'SMP',
    '11th': 'SMP',
    '7th-8th': 'SD',
    '9th': 'SD',
    '12th': 'SMP',
    '1st-4th': 'TK',
    '5th-6th': 'TK'
}
dftest['Pendidikan'] = dftest['Pendidikan'].replace(pddterakhir)
dftest['Pekerjaan'].replace("?", float("NaN"), inplace=True)
# dftest.dropna(subset=["Pekerjaan"], inplace=True)
dftest = pd.get_dummies(dftest, columns=['Pendidikan', 'Pekerjaan'])
dftest = dftest.drop(['Berat Akhir', 'Keuntungan Kapital', 'Kerugian Capital'], axis=1)

submission = model.predict(dftest)
dftest['Gaji'] = submission
df_out = dftest[['id', 'Gaji']]
# df_out

from google.colab import files
df_out.to_csv('submission.csv', index=False) 
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>